In [4]:
import pandas as pd
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.load_model("./CLF_model.txt")

XGBoostError: [10:51:04] C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:866: Check failed: fi->Read(&mparam_, sizeof(mparam_)) == sizeof(mparam_) (0 vs. 136) : BoostLearner: wrong model format

In [ ]:
# Import libraries
import string
import nltk
import pandas as pd
import numpy as np
import spacy as sp
from nltk.corpus import stopwords
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import sys
!{sys.executable} -m pip install xgboost

nltk.download('stopwords')

In [2]:
NYTwebsite_text = """On his first presidential visit to Saudi Arabia, Mr. Biden received an unsmiling fist bump from the crown prince before meeting King Salman. Their body language is being parsed amid tensions over the Saudi human rights record. President Biden used his visit to Israel this week to bolster the blossoming relationship between Israel and a handful of Arab states. But it is Mr. Biden’s own relationship with Saudi Arabia that looms largest over the second part of his four-day trip, and the kingdom’s welcome of him was being closely watched as an indication of how the overall visit might unfold.

As Air Force One landed, just before 6 p.m. local time, the Saudi reception, while led by a senior member of the royal family, was decidedly lacking in pomp. Lining the lilac carpet were a small number of uniformed security officers bearing swords, but even fewer than were dispatched to welcome President Barack Obama when he arrived in the kingdom to a chilly reception in 2016.

Mr. Biden emerged from the plane and made his way down the stairs for his first visit to Saudi Arabia as president, wearing his aviator shades, waving and looking serious. He was greeted by Princess Reema bint Bandar Al Saud, the Saudi ambassador to Washington, and Prince Khalid Al Faisal, a senior member of the royal family who is the governor of Mecca and is close to King Salman.

The president climbed into his limousine just two minutes after disembarking. He was  taken directly to meet the king.

Arriving at the al-Salam palace, Mr. Biden was met by Crown Prince Mohammed bin Salman, and the two exchanged an apparently wordless fist bump before the prince led the president inside.

The Saudi-owned Al Arabiya satellite channel broadcast images of Mr. Biden being warmly welcomed by King Salman, and shaking his hand. Other images showed Mr. Biden and the king sitting next to each other in armchairs in an ornate room during a meeting, with other U.S. and Saudi officials, including Prince Mohammed, sitting nearby.

Later, Mr. Biden is to hold a “working session” along with his team with Prince Mohammed and various Saudi ministers.Biden’s arrival in Saudi Arabia was polite but perfunctory compared with the enthusiastic greeting President Donald J. Trump received in 2017 when King Salman welcomed him on the tarmac. Mr. Trump made Saudi Arabia his first overseas destination as president, a sharp break with tradition and a sign of how closely he would align his administration with the Saudis.

Even though Mr. Biden vowed as a candidate to punish Saudi Arabia over Mr. Khashoggi’s murder by making the kingdom a “pariah,” the president decided that it was worth the political cost of traveling there this week to counter Chinese influence, press for additional oil production and encourage closer ties with Israel.

In other cases, recently including Cuba and Venezuela, Mr. Biden has stressed that his administration is making democracy and respect for human rights the paramount consideration for dealing with other nations’ leaders. But on Thursday in Jerusalem, he said he was going to Saudi Arabia in order to promote U.S. interests.

Those include getting the kingdom to pump more oil from its somewhat modest spare capacity."""
dict = {'label':['unknown'],
        'text':[NYTwebsite_text],
       }
data = pd.DataFrame(dict)

def text_preprocess(message):
    # Remove punctuations
    nopunc = [char for char in message if char not in string.punctuation]

    # Join the characters again
    nopunc = "".join(nopunc)
    nopunc = nopunc.lower()

    # Remove any stopwords and characters that arent letters
    nostop = [
        word
        for word in nopunc.split()
        if word.lower() not in stopwords.words("english") and word.isalpha()
    ]
    return nostop


false_text = data[data["label"] == "fake"]["text"]
true_text = data[data["label"] == "true"]["text"]
print(f"Number of fake messages: {len(false_text)}")
print(f"Number of true messages: {len(true_text)}")

data["text"] = data["text"].apply(text_preprocess)

####################################################################### TAKES FOREVER TO PROSESS
from sklearn.utils import shuffle
data_shuffled = data.sample(frac=1).reset_index(drop=True)
print(data_shuffled.head(10))


# Remove punctuations/stopwords from all messages
data_shuffled["text"] = data_shuffled["text"].apply(text_preprocess)
data_shuffled.head(20)

# Convert messages (as lists of string tokens) to strings
data_shuffled["text"] = data_shuffled["text"].agg(lambda x: " ".join(map(str, x)))
data_shuffled.head()

# Initialize count vectorizer
vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(data_shuffled["text"])

# Fetch the vocabulary set
print(f"20 BOW Features: {vectorizer.get_feature_names()[20:40]}")
print(f"Total number of vocab words: {len(vectorizer.vocabulary_)}")

# Convert strings to vectors using BoW
dataset1_bow = bow_transformer.transform(data_shuffled["text"])

# Print the shape of the sparse matrix and count the number of non-zero occurrences
print(f"Shape of sparse matrix: {dataset1_bow.shape}")
print(f"Amount of non-zero occurrences: {dataset1_bow.nnz}")

# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer().fit(dataset1_bow)

#Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(dataset1_bow)
print(messages_tfidf.shape)

# Make predictions
predict_train = clf.predict(messages_tfidf)

# an example prediction
print(
    "predicted:",
    clf.predict(
        tfidf_transformer.transform(bow_transformer.transform([data_shuffled["text"][9]]))
    )[0],
)
print("expected:", data_shuffled["label"][9])

# print the overall accuracy of the model
label_predictions = clf.predict(msg_test)
print(f"Accuracy of the model: {metrics.accuracy_score(label_test, label_predictions):0.3f}")
# Transform entire BoW into tf-idf corpus
messages_tfidf = tfidf_transformer.transform(dataset1_bow)
print(messages_tfidf.shape)




NameError: name 'pd' is not defined